In [2]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools

In [3]:
from sklearn.model_selection import StratifiedKFold
dataset=pd.read_csv("iris-setosa.csv")

X = dataset.columns[1:3]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)

In [4]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=90.0):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

#geometric mean
def score(y_predict,y_test):
    test_min=0
    test_max=0
    pred_min=0
    pred_max=0
    y_test=np.asarray(y_test)
    for i in range(0,len(y_test)):
        if(y_test[i]==1):
            test_min=test_min+1
        else:
            test_max=test_max+1
    print("y_test min",test_min)       
    print("y_test max",test_max)
    for i in range(0,len(y_predict)):
        if(y_predict[i]==1 and y_predict[i]==y_test[i]):
            pred_min=pred_min+1
        elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
            pred_max=pred_max+1
    print("y_pred min",pred_min)       
    print("y_pred max",pred_max)
    se=pred_min/test_min
    sp=pred_max/test_max
    print(se,sp)
    gm=math.sqrt(se*sp)
    print("GM",gm)   

In [5]:
from cvxopt import matrix
class HYP_SVM(object):

    #geometric mean
    def score(y_predict,y_test):
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        print("y_test min",test_min)       
        print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        print("y_pred min",pred_min)       
        print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm)    
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        self.sigma = sigma
        if self.C is not None: self.C = float(self.C)
            
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma, "kernel": self.kernel}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train, y):
        n_samples, n_features = X_train.shape 
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)
        typ=1
        if(typ==1):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(typ==2):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=103/4074
        r_min=1
        self.m=func[0:115]*r_min
        print(self.m.shape)
        self.m=np.append(self.m,func[115:5473]*r_max)
        print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 
        #nt_samples, nt_features = X_test.shape
        # Gram matrix

        print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None        
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                y_predict[i] = s
                print(y_predict[i]+self.b)
                #print(self.b)
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [7]:
if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        C_vals = [1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4]
        sigma = [9e-2, 9e-1, 9e+1, 9e+2, 9e+3, 9e+4]
        combs = list(itertools.product(C_vals, sigma))
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        skf2 = StratifiedKFold(n_splits=len(combs), shuffle=True, random_state=1)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            i=0
            for train_index, test_index in skf2.split(X_train, y_train):
                print("\n\n Using C=", combs[i][0], "and sigma=", combs[i][1])
                clf = HYP_SVM(C=combs[i][0], sigma=combs[i][1])
                #clf = HYP_SVM(C=combs[i][0], kernel=gaussian_kernel, sigma=combs[i][1])
                i+=1
                #typ = 2            
                
                #print(len(X_train), len(X_test), len(y_train))
                clf.m_func(X_train,y_train)
                
                clf.fit(X_train, y_train)
                y_predict = clf.predict(X_test)
                print(y_test)
                score(y_predict,y_test)
                correct = np.sum(y_predict == y_test)
                mse = mean_squared_error(y_test, y_predict)
                print("%d out of %d predictions correct" % (correct, len(y_predict)))
                print("Accuracy",correct/len(y_predict))
                print("Errore quadratico medio: ", mse)
            
    hyp_svm()  



 Using C= 0.01 and sigma= 0.09
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -1.8532e+01 -8.3103e+00  5e+02  2e+01  3e-16
 1: -7.2111e+00 -3.0922e+00  4e+01  2e+00  4e-16
 2: -8.0984e-01 -2.2334e+00  2e+00  2e-02  1e-15
 3: -7.8553e-01 -8.5733e-01  7e-02  2e-04  4e-16
 4: -7.9153e-01 -7.9821e-01  7e-03  2e-05  4e-16
 5: -7.9264e-01 -7.9355e-01  9e-04  1e-06  3e-16
 6: -7.9283e-01 -7.9296e-01  1e-04  6e-08  3e-16
 7: -7.9285e-01 -7.9287e-01  1e-05  2e-09  3e-16
 8: -7.9286e-01 -7.9286e-01  8e-07  5e-11  3e-16
 9: -7.9286e-01 -7.9286e-01  4e-08  7e-13  4e-16
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -1.8532e+01 -8.3103e+00  5e+02  2e+01  3e-16
 1: -7.2111e+00 -3.0922e+00  4e+01  2e+00  4e-16
 2: -8.0984e-01 -2.2334e+00  2e+00  2e-02  1e-15
 3: -7.8553e-01 -8.5733e-01  7e-02  2e-04  4e-16
 4: -7.9153e-01 -7.9821e-01  7e-03  2e-05  4e-16
 5: -7.9264e-01 -7.9355e-01  9e-04  1e-06  3e-16
 6: -7.9283e-01 -7.

(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.3867e+01 -2.5347e+00  6e+02  3e+01  3e-15
 1: -1.3258e+00 -2.5227e+00  8e+00  3e-01  4e-15
 2: -7.8964e-01 -1.7596e+00  1e+00  4e-17  8e-16
 3: -7.9977e-01 -8.1102e-01  1e-02  7e-17  7e-16
 4: -8.0000e-01 -8.0011e-01  1e-04  5e-17  6e-16
 5: -8.0000e-01 -8.0000e-01  1e-06  3e-16  5e-16
 6: -8.0000e-01 -8.0000e-01  2e-08  2e-16  7e-16
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.3867e+01 -2.5347e+00  6e+02  3e+01  3e-15
 1: -1.3258e+00 -2.5227e+00  8e+00  3e-01  4e-15
 2: -7.8964e-01 -1.7596e+00  1e+00  4e-17  8e-16
 3: -7.9977e-01 -8.1102e-01  1e-02  7e-17  7e-16
 4: -8.0000e-01 -8.0011e-01  1e-04  5e-17  6e-16
 5: -8.0000e-01 -8.0000e-01  1e-06  3e-16  5e-16
 6: -8.0000e-01 -8.0000e-01  2e-08  2e-16  7e-16
Optimal solution found.
optimal
-0.3333333323987813
-0.33333332634835877
-0.3333333280744276
-0.33333332800890897
-0.33333332982054703
-0.333333327093

(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.8664e+01 -2.5467e+01  6e+02  2e+01  4e-15
 1: -8.3920e+00 -2.4302e+01  2e+01  2e-01  4e-15
 2: -7.9459e+00 -8.8054e+00  9e-01  9e-16  1e-15
 3: -7.9994e+00 -8.0082e+00  9e-03  1e-15  1e-15
 4: -8.0000e+00 -8.0001e+00  1e-04  6e-16  1e-15
 5: -8.0000e+00 -8.0000e+00  2e-05  3e-15  1e-15
 6: -8.0000e+00 -8.0000e+00  4e-06  3e-15  2e-15
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.8664e+01 -2.5467e+01  6e+02  2e+01  4e-15
 1: -8.3920e+00 -2.4302e+01  2e+01  2e-01  4e-15
 2: -7.9459e+00 -8.8054e+00  9e-01  9e-16  1e-15
 3: -7.9994e+00 -8.0082e+00  9e-03  1e-15  1e-15
 4: -8.0000e+00 -8.0001e+00  1e-04  6e-16  1e-15
 5: -8.0000e+00 -8.0000e+00  2e-05  3e-15  1e-15
 6: -8.0000e+00 -8.0000e+00  4e-06  3e-15  2e-15
Optimal solution found.
optimal
-0.33333157135592567
-0.3333278134900738
-0.3333287345517699
-0.3333296137927446
-0.3333303900564779
-0.33332885843911

(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.7426e+02 -3.8449e+03  3e+03  1e-15  4e-14
 1: -6.5761e+02 -9.2331e+02  3e+02  2e-13  6e-14
 2: -7.6853e+02 -8.1317e+02  4e+01  2e-14  6e-14
 3: -7.8485e+02 -7.9262e+02  8e+00  7e-14  7e-14
 4: -7.8902e+02 -7.9027e+02  1e+00  2e-13  7e-14
 5: -7.8981e+02 -7.8988e+02  6e-02  1e-13  7e-14
 6: -7.8985e+02 -7.8986e+02  7e-04  2e-13  8e-14
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.7426e+02 -3.8449e+03  3e+03  1e-15  4e-14
 1: -6.5761e+02 -9.2331e+02  3e+02  2e-13  6e-14
 2: -7.6853e+02 -8.1317e+02  4e+01  2e-14  6e-14
 3: -7.8485e+02 -7.9262e+02  8e+00  7e-14  7e-14
 4: -7.8902e+02 -7.9027e+02  1e+00  2e-13  7e-14
 5: -7.8981e+02 -7.8988e+02  6e-02  1e-13  7e-14
 6: -7.8985e+02 -7.8986e+02  7e-04  2e-13  8e-14
Optimal solution found.
optimal
-0.31282359570382273
-0.2828976231625055
-0.28977521874271045
-0.30104370406641995
-0.30585996810191735
-0.29403172838

(120, 120)
     pcost       dcost       gap    pres   dres
 0:  2.1725e+04 -2.2554e+05  4e+05  1e-01  4e-14
 1:  1.2326e+04 -2.6641e+04  5e+04  1e-02  5e-14
 2:  2.3237e+03 -3.0579e+03  5e+03  1e-13  3e-14
 3:  3.0650e+02 -4.4096e+02  7e+02  3e-14  2e-14
 4:  3.0290e+01 -7.4418e+01  1e+02  8e-15  8e-15
 5: -1.0084e+00 -2.7268e+01  3e+01  5e-15  4e-15
 6: -1.3198e+01 -2.2240e+01  9e+00  6e-15  3e-15
 7: -1.6838e+01 -1.8960e+01  2e+00  2e-15  3e-15
 8: -1.7964e+01 -1.8194e+01  2e-01  1e-14  3e-15
 9: -1.8056e+01 -1.8082e+01  3e-02  1e-14  3e-15
10: -1.8068e+01 -1.8068e+01  3e-04  8e-15  3e-15
11: -1.8068e+01 -1.8068e+01  3e-06  2e-14  3e-15
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0:  2.1725e+04 -2.2554e+05  4e+05  1e-01  4e-14
 1:  1.2326e+04 -2.6641e+04  5e+04  1e-02  5e-14
 2:  2.3237e+03 -3.0579e+03  5e+03  1e-13  3e-14
 3:  3.0650e+02 -4.4096e+02  7e+02  3e-14  2e-14
 4:  3.0290e+01 -7.4418e+01  1e+02  8e-15  8e-15
 5:

...

(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.3264e+05 -1.3359e+09  1e+09  6e-11  3e-11
 1: -5.3285e+05 -1.4151e+07  1e+07  3e-11  5e-11
 2: -5.5246e+05 -9.3415e+05  4e+05  4e-11  5e-11
 3: -7.9584e+05 -8.1394e+05  2e+04  1e-10  7e-11
 4: -7.9814e+05 -8.0103e+05  3e+03  3e-10  7e-11
 5: -7.9869e+05 -7.9989e+05  1e+03  3e-12  7e-11
 6: -7.9900e+05 -7.9937e+05  4e+02  2e-10  7e-11
 7: -7.9908e+05 -7.9928e+05  2e+02  1e-10  6e-11
 8: -7.9912e+05 -7.9923e+05  1e+02  1e-10  7e-11
 9: -7.9916e+05 -7.9919e+05  3e+01  4e-11  6e-11
10: -7.9917e+05 -7.9918e+05  3e+00  5e-11  7e-11
11: -7.9917e+05 -7.9917e+05  4e-02  2e-11  7e-11
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.3264e+05 -1.3359e+09  1e+09  6e-11  3e-11
 1: -5.3285e+05 -1.4151e+07  1e+07  3e-11  5e-11
 2: -5.5246e+05 -9.3415e+05  4e+05  4e-11  5e-11
 3: -7.9584e+05 -8.1394e+05  2e+04  1e-10  7e-11
 4: -7.9814e+05 -8.0103e+05  3e+03  3e-10  7e-11
 5:

(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.3611e+01 -2.6173e+00  6e+02  3e+01  3e-15
 1: -1.3246e+00 -2.5224e+00  8e+00  3e-01  4e-15
 2: -7.8950e-01 -1.7594e+00  1e+00  3e-16  7e-16
 3: -7.9975e-01 -8.1105e-01  1e-02  2e-17  7e-16
 4: -7.9998e-01 -8.0014e-01  2e-04  6e-17  5e-16
 5: -7.9998e-01 -8.0001e-01  3e-05  2e-17  7e-16
 6: -7.9999e-01 -7.9999e-01  6e-06  1e-16  5e-16
 7: -7.9999e-01 -7.9999e-01  1e-06  2e-16  4e-16
 8: -7.9999e-01 -7.9999e-01  5e-07  1e-16  4e-16
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -5.3611e+01 -2.6173e+00  6e+02  3e+01  3e-15
 1: -1.3246e+00 -2.5224e+00  8e+00  3e-01  4e-15
 2: -7.8950e-01 -1.7594e+00  1e+00  3e-16  7e-16
 3: -7.9975e-01 -8.1105e-01  1e-02  2e-17  7e-16
 4: -7.9998e-01 -8.0014e-01  2e-04  6e-17  5e-16
 5: -7.9998e-01 -8.0001e-01  3e-05  2e-17  7e-16
 6: -7.9999e-01 -7.9999e-01  6e-06  1e-16  5e-16
 7: -7.9999e-01 -7.9999e-01  1e-06  2e-16  4e-16
 8:

(120, 120)
     pcost       dcost       gap    pres   dres
 0: -6.9721e+00 -2.0377e+01  5e+02  2e+01  6e-16
 1: -3.2713e+00 -1.8825e+01  2e+01  2e-01  6e-16
 2: -3.3298e+00 -5.2750e+00  2e+00  2e-02  4e-16
 3: -3.7183e+00 -4.1193e+00  4e-01  2e-03  4e-16
 4: -3.8592e+00 -3.9358e+00  8e-02  2e-04  4e-16
 5: -3.8860e+00 -3.9014e+00  2e-02  3e-05  4e-16
 6: -3.8925e+00 -3.8934e+00  9e-04  1e-06  5e-16
 7: -3.8929e+00 -3.8929e+00  2e-05  3e-08  4e-16
 8: -3.8929e+00 -3.8929e+00  2e-07  3e-10  5e-16
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0: -6.9721e+00 -2.0377e+01  5e+02  2e+01  6e-16
 1: -3.2713e+00 -1.8825e+01  2e+01  2e-01  6e-16
 2: -3.3298e+00 -5.2750e+00  2e+00  2e-02  4e-16
 3: -3.7183e+00 -4.1193e+00  4e-01  2e-03  4e-16
 4: -3.8592e+00 -3.9358e+00  8e-02  2e-04  4e-16
 5: -3.8860e+00 -3.9014e+00  2e-02  3e-05  4e-16
 6: -3.8925e+00 -3.8934e+00  9e-04  1e-06  5e-16
 7: -3.8929e+00 -3.8929e+00  2e-05  3e-08  4e-16
 8:

(120, 120)
     pcost       dcost       gap    pres   dres
 0:  4.8616e+02 -3.5972e+03  4e+03  1e-14  3e-15
 1:  9.1436e+01 -4.0837e+02  5e+02  1e-13  2e-15
 2: -1.6625e+01 -6.9210e+01  5e+01  8e-14  9e-16
 3: -2.5949e+01 -3.0590e+01  5e+00  7e-15  3e-16
 4: -2.6279e+01 -2.6917e+01  6e-01  1e-14  2e-16
 5: -2.6329e+01 -2.6375e+01  5e-02  6e-15  2e-16
 6: -2.6336e+01 -2.6338e+01  2e-03  1e-14  2e-16
 7: -2.6337e+01 -2.6337e+01  6e-05  1e-14  2e-16
 8: -2.6337e+01 -2.6337e+01  3e-06  2e-16  2e-16
Optimal solution found.
optimal
(115,)
(120,)
(120, 120)
     pcost       dcost       gap    pres   dres
 0:  4.8616e+02 -3.5972e+03  4e+03  1e-14  3e-15
 1:  9.1436e+01 -4.0837e+02  5e+02  1e-13  2e-15
 2: -1.6625e+01 -6.9210e+01  5e+01  8e-14  9e-16
 3: -2.5949e+01 -3.0590e+01  5e+00  7e-15  3e-16
 4: -2.6279e+01 -2.6917e+01  6e-01  1e-14  2e-16
 5: -2.6329e+01 -2.6375e+01  5e-02  6e-15  2e-16
 6: -2.6336e+01 -2.6338e+01  2e-03  1e-14  2e-16
 7: -2.6337e+01 -2.6337e+01  6e-05  1e-14  2e-16
 8:

KeyboardInterrupt: 